In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, plot_confusion_matrix
from sklearn.decomposition import PCA
import numpy as np

In [2]:
from dataset import HARDatasetCrops

## Data

### Features

In [3]:
dataset = HARDatasetCrops('motionsense-dataset/train', 256, 50, 50)

In [4]:
print('Dataset size:', len(dataset))

sample, _ = dataset[0]
print('Datapoints shape:', sample.shape)

Dataset size: 4095
Datapoints shape: (256, 12)


Our dataset is composed of 5205 datapoints, each have shape `(256, 12)` because we have 12 signals of 256 samples each.

For our base line model we will use the **user-acceleration**.

In [10]:
dataset = HARDatasetCrops('motionsense-dataset/train', 256, 50, 50, metadata_file='motionsense-dataset/data_subjects_info.csv')

In [11]:
X = np.array([sample[:,-3:] for sample, _, _ in dataset])

In [12]:
metadata = np.array([metadata for _, _, metadata in dataset])
metadata = metadata.reshape((-1, 4, 1))

### Classes

In order to fit a `sklearn` model we should encode each class with a unique integer:

In [7]:
label_encoder = sklearn.preprocessing.LabelEncoder()
label_encoder.fit(list(dataset.CLASSES.keys()));

In [8]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [9]:
y = to_categorical(label_encoder.transform([cls for _, cls in dataset]))

Train-test splitting

In [13]:
indices = np.random.choice(np.arange(X.shape[0]), X.shape[0], replace=False)

In [14]:
train_indices = indices[:int(X.shape[0] * 0.8)]
test_indices = indices[int(X.shape[0] * 0.8):]

In [15]:
X_train, y_train = X[train_indices], y[train_indices]
X_test, y_test = X[test_indices], y[test_indices]
metadata_train = metadata[train_indices]
metadata_test = metadata[test_indices]

In [16]:
print("X train shape: {}", X_train.shape)
print("y train shape: {}", y_train.shape)
print("X test shape: {}", X_test.shape)
print("y test shape: {}", y_test.shape)
print("metdata train shape: {}", metadata_train.shape)
print("metdata test shape: {}", metadata_test.shape)

X train shape: {} (3276, 256, 3)
y train shape: {} (3276, 6)
X test shape: {} (819, 256, 3)
y test shape: {} (819, 6)
metdata train shape: {} (3276, 4, 1)
metdata test shape: {} (819, 4, 1)


In [17]:
print('Number of training examples:', y_train.shape[0])
print('Number of testing examples:', y_test.shape[0])

Number of training examples: 3276
Number of testing examples: 819


## CNN

In [18]:
import keras

In [19]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, Flatten, concatenate, Input

In [20]:
#create model
clf = Sequential()
#add model layers
clf.add(Conv1D(16, kernel_size=5, activation="relu", input_shape=(256, 3)))
clf.add(Conv1D(32, kernel_size=5, activation="relu"))
clf.add(Flatten())
clf.add(Dense(6, activation="softmax", kernel_regularizer=keras.regularizers.l2(0.01)))

In [21]:
clf.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [22]:
clf.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30)

Train on 3276 samples, validate on 819 samples
Epoch 1/30
3276/3276 [==============================] - 1s 392us/step - loss: 1.1588 - accuracy: 0.5607 - val_loss: 0.8419 - val_accuracy: 0.7131
Epoch 2/30
3276/3276 [==============================] - 1s 272us/step - loss: 0.7115 - accuracy: 0.7579 - val_loss: 0.7180 - val_accuracy: 0.7521
Epoch 3/30
3276/3276 [==============================] - 1s 264us/step - loss: 0.6017 - accuracy: 0.8053 - val_loss: 0.7178 - val_accuracy: 0.7582
Epoch 4/30
3276/3276 [==============================] - 1s 264us/step - loss: 0.5390 - accuracy: 0.8303 - val_loss: 0.6707 - val_accuracy: 0.7827
Epoch 5/30
3276/3276 [==============================] - 1s 268us/step - loss: 0.5076 - accuracy: 0.8404 - val_loss: 0.7111 - val_accuracy: 0.7778
Epoch 6/30
3276/3276 [==============================] - 1s 264us/step - loss: 0.4931 - accuracy: 0.8452 - val_loss: 0.6275 - val_accuracy: 0.8230
Epoch 7/30
3276/3276 [==============================] - 1s 262us/step - loss:

In [23]:
print(classification_report(np.argmax(y_train, axis=1), np.argmax(clf.predict(X_train), axis=1), target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         dws       1.00      0.99      0.99       284
         jog       1.00      1.00      1.00       307
         sit       0.87      0.97      0.91       794
         std       0.96      0.84      0.89       726
         ups       0.99      1.00      0.99       343
         wlk       1.00      0.99      0.99       822

    accuracy                           0.95      3276
   macro avg       0.97      0.96      0.96      3276
weighted avg       0.96      0.95      0.95      3276



In [24]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(clf.predict(X_test), axis=1), target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         dws       0.81      0.77      0.79        75
         jog       0.96      0.95      0.95        77
         sit       0.89      0.94      0.92       210
         std       0.91      0.86      0.89       174
         ups       0.76      0.69      0.72        93
         wlk       0.86      0.91      0.88       190

    accuracy                           0.87       819
   macro avg       0.87      0.85      0.86       819
weighted avg       0.87      0.87      0.87       819



## Metadata

In [25]:
# create model
conv = Sequential()
# add model layers
conv.add(Conv1D(16, kernel_size=5, activation="relu", input_shape=(256, 3)))
conv.add(Conv1D(32, kernel_size=5, activation="relu"))
conv.add(Flatten(name="coefs"))

metadata_input_tensor = Input(shape=(4, 1))
metadata_input = Flatten(name="flatten")(metadata_input_tensor)

last_layer = conv.get_layer("coefs").output
x = concatenate([last_layer, metadata_input], axis=1)
out = Dense(6, activation="softmax")(x)

clf = Model([conv.input, metadata_input_tensor], out)

In [26]:
clf.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [27]:
clf.fit([X_train, metadata_train], y_train, validation_data=([X_test, metadata_test], y_test), epochs=30)

Train on 3276 samples, validate on 819 samples
Epoch 1/30
3276/3276 [==============================] - 1s 374us/step - loss: 0.9303 - accuracy: 0.6242 - val_loss: 0.6766 - val_accuracy: 0.7436
Epoch 2/30
3276/3276 [==============================] - 1s 312us/step - loss: 0.5016 - accuracy: 0.7866 - val_loss: 0.5411 - val_accuracy: 0.7753
Epoch 3/30
3276/3276 [==============================] - 1s 307us/step - loss: 0.3788 - accuracy: 0.8483 - val_loss: 0.5021 - val_accuracy: 0.8010
Epoch 4/30
3276/3276 [==============================] - 1s 302us/step - loss: 0.2967 - accuracy: 0.8886 - val_loss: 0.4907 - val_accuracy: 0.8291
Epoch 5/30
3276/3276 [==============================] - 1s 309us/step - loss: 0.2517 - accuracy: 0.9106 - val_loss: 0.4769 - val_accuracy: 0.8535
Epoch 6/30
3276/3276 [==============================] - 1s 305us/step - loss: 0.2101 - accuracy: 0.9216 - val_loss: 0.4639 - val_accuracy: 0.8474
Epoch 7/30
3276/3276 [==============================] - 1s 311us/step - loss:

In [28]:
print(classification_report(np.argmax(y_train, axis=1), np.argmax(clf.predict([X_train, metadata_train]), axis=1), target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         dws       1.00      1.00      1.00       284
         jog       1.00      1.00      1.00       307
         sit       0.97      0.99      0.98       794
         std       0.99      0.96      0.98       726
         ups       1.00      1.00      1.00       343
         wlk       1.00      1.00      1.00       822

    accuracy                           0.99      3276
   macro avg       0.99      0.99      0.99      3276
weighted avg       0.99      0.99      0.99      3276



In [29]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(clf.predict([X_test, metadata_test]), axis=1), target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         dws       0.75      0.68      0.71        75
         jog       0.91      0.96      0.94        77
         sit       0.98      0.98      0.98       210
         std       0.97      0.97      0.97       174
         ups       0.68      0.66      0.67        93
         wlk       0.83      0.85      0.84       190

    accuracy                           0.88       819
   macro avg       0.85      0.85      0.85       819
weighted avg       0.88      0.88      0.88       819



## All signals

In [34]:
X = np.array([sample for sample, _, _ in dataset])

X_train = X[train_indices]
X_test = X[test_indices]

In [36]:
print("X train shape: {}", X_train.shape)
print("y train shape: {}", y_train.shape)
print("X test shape: {}", X_test.shape)
print("y test shape: {}", y_test.shape)
print("metdata train shape: {}", metadata_train.shape)
print("metdata test shape: {}", metadata_test.shape)

X train shape: {} (3276, 256, 12)
y train shape: {} (3276, 6)
X test shape: {} (819, 256, 12)
y test shape: {} (819, 6)
metdata train shape: {} (3276, 4, 1)
metdata test shape: {} (819, 4, 1)


In [20]:
#create model
clf = Sequential()
#add model layers
clf.add(Conv1D(16, kernel_size=5, activation="relu", input_shape=(256, 12)))
clf.add(Conv1D(32, kernel_size=5, activation="relu"))
clf.add(Flatten())
clf.add(Dense(6, activation="softmax", kernel_regularizer=keras.regularizers.l2(0.01)))

In [21]:
clf.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [22]:
clf.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30)

Train on 3276 samples, validate on 819 samples
Epoch 1/30
3276/3276 [==============================] - 1s 392us/step - loss: 1.1588 - accuracy: 0.5607 - val_loss: 0.8419 - val_accuracy: 0.7131
Epoch 2/30
3276/3276 [==============================] - 1s 272us/step - loss: 0.7115 - accuracy: 0.7579 - val_loss: 0.7180 - val_accuracy: 0.7521
Epoch 3/30
3276/3276 [==============================] - 1s 264us/step - loss: 0.6017 - accuracy: 0.8053 - val_loss: 0.7178 - val_accuracy: 0.7582
Epoch 4/30
3276/3276 [==============================] - 1s 264us/step - loss: 0.5390 - accuracy: 0.8303 - val_loss: 0.6707 - val_accuracy: 0.7827
Epoch 5/30
3276/3276 [==============================] - 1s 268us/step - loss: 0.5076 - accuracy: 0.8404 - val_loss: 0.7111 - val_accuracy: 0.7778
Epoch 6/30
3276/3276 [==============================] - 1s 264us/step - loss: 0.4931 - accuracy: 0.8452 - val_loss: 0.6275 - val_accuracy: 0.8230
Epoch 7/30
3276/3276 [==============================] - 1s 262us/step - loss:

In [23]:
print(classification_report(np.argmax(y_train, axis=1), np.argmax(clf.predict(X_train), axis=1), target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         dws       1.00      0.99      0.99       284
         jog       1.00      1.00      1.00       307
         sit       0.87      0.97      0.91       794
         std       0.96      0.84      0.89       726
         ups       0.99      1.00      0.99       343
         wlk       1.00      0.99      0.99       822

    accuracy                           0.95      3276
   macro avg       0.97      0.96      0.96      3276
weighted avg       0.96      0.95      0.95      3276



In [24]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(clf.predict(X_test), axis=1), target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         dws       0.81      0.77      0.79        75
         jog       0.96      0.95      0.95        77
         sit       0.89      0.94      0.92       210
         std       0.91      0.86      0.89       174
         ups       0.76      0.69      0.72        93
         wlk       0.86      0.91      0.88       190

    accuracy                           0.87       819
   macro avg       0.87      0.85      0.86       819
weighted avg       0.87      0.87      0.87       819



## Metadata

In [25]:
# create model
conv = Sequential()
# add model layers
conv.add(Conv1D(16, kernel_size=5, activation="relu", input_shape=(256, 3)))
conv.add(Conv1D(32, kernel_size=5, activation="relu"))
conv.add(Flatten(name="coefs"))

metadata_input_tensor = Input(shape=(4, 1))
metadata_input = Flatten(name="flatten")(metadata_input_tensor)

last_layer = conv.get_layer("coefs").output
x = concatenate([last_layer, metadata_input], axis=1)
out = Dense(6, activation="softmax")(x)

clf = Model([conv.input, metadata_input_tensor], out)

In [26]:
clf.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [27]:
clf.fit([X_train, metadata_train], y_train, validation_data=([X_test, metadata_test], y_test), epochs=30)

Train on 3276 samples, validate on 819 samples
Epoch 1/30
3276/3276 [==============================] - 1s 374us/step - loss: 0.9303 - accuracy: 0.6242 - val_loss: 0.6766 - val_accuracy: 0.7436
Epoch 2/30
3276/3276 [==============================] - 1s 312us/step - loss: 0.5016 - accuracy: 0.7866 - val_loss: 0.5411 - val_accuracy: 0.7753
Epoch 3/30
3276/3276 [==============================] - 1s 307us/step - loss: 0.3788 - accuracy: 0.8483 - val_loss: 0.5021 - val_accuracy: 0.8010
Epoch 4/30
3276/3276 [==============================] - 1s 302us/step - loss: 0.2967 - accuracy: 0.8886 - val_loss: 0.4907 - val_accuracy: 0.8291
Epoch 5/30
3276/3276 [==============================] - 1s 309us/step - loss: 0.2517 - accuracy: 0.9106 - val_loss: 0.4769 - val_accuracy: 0.8535
Epoch 6/30
3276/3276 [==============================] - 1s 305us/step - loss: 0.2101 - accuracy: 0.9216 - val_loss: 0.4639 - val_accuracy: 0.8474
Epoch 7/30
3276/3276 [==============================] - 1s 311us/step - loss:

In [28]:
print(classification_report(np.argmax(y_train, axis=1), np.argmax(clf.predict([X_train, metadata_train]), axis=1), target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         dws       1.00      1.00      1.00       284
         jog       1.00      1.00      1.00       307
         sit       0.97      0.99      0.98       794
         std       0.99      0.96      0.98       726
         ups       1.00      1.00      1.00       343
         wlk       1.00      1.00      1.00       822

    accuracy                           0.99      3276
   macro avg       0.99      0.99      0.99      3276
weighted avg       0.99      0.99      0.99      3276



In [29]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(clf.predict([X_test, metadata_test]), axis=1), target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         dws       0.75      0.68      0.71        75
         jog       0.91      0.96      0.94        77
         sit       0.98      0.98      0.98       210
         std       0.97      0.97      0.97       174
         ups       0.68      0.66      0.67        93
         wlk       0.83      0.85      0.84       190

    accuracy                           0.88       819
   macro avg       0.85      0.85      0.85       819
weighted avg       0.88      0.88      0.88       819



## LSTM

In [39]:
X = np.array([sample[:,-3:] for sample, _, _ in dataset])

X_train, y_train = X[train_indices], y[train_indices]
X_test, y_test = X[test_indices], y[test_indices]
metadata_train = metadata[train_indices]
metadata_test = metadata[test_indices]

In [40]:
from keras.layers import LSTM

In [44]:
#create model
clf = Sequential()
#add model layers
clf.add(LSTM(128, input_shape=(256, 3)))
clf.add(Dense(6, activation="softmax", kernel_regularizer=keras.regularizers.l2(0.01)))

In [45]:
clf.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [46]:
clf.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30)

Train on 3276 samples, validate on 819 samples
Epoch 1/30
3276/3276 [==============================] - 26s 8ms/step - loss: 1.6347 - accuracy: 0.3495 - val_loss: 1.5116 - val_accuracy: 0.4322
Epoch 2/30
3276/3276 [==============================] - 25s 8ms/step - loss: 1.1245 - accuracy: 0.5256 - val_loss: 0.9684 - val_accuracy: 0.5592
Epoch 3/30
3276/3276 [==============================] - 25s 8ms/step - loss: 1.3953 - accuracy: 0.4359 - val_loss: 1.2896 - val_accuracy: 0.4737
Epoch 4/30
3276/3276 [==============================] - 25s 8ms/step - loss: 1.2004 - accuracy: 0.5015 - val_loss: 1.2714 - val_accuracy: 0.4713
Epoch 5/30
3276/3276 [==============================] - 25s 8ms/step - loss: 1.1137 - accuracy: 0.4973 - val_loss: 1.0797 - val_accuracy: 0.5250
Epoch 6/30
3276/3276 [==============================] - 25s 8ms/step - loss: 1.0063 - accuracy: 0.5598 - val_loss: 1.2686 - val_accuracy: 0.5531
Epoch 7/30
3276/3276 [==============================] - 27s 8ms/step - loss: 1.2043

In [47]:
print(classification_report(np.argmax(y_train, axis=1), np.argmax(clf.predict(X_train), axis=1), target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         dws       0.00      0.00      0.00       284
         jog       1.00      0.28      0.44       307
         sit       0.35      1.00      0.52       794
         std       0.00      0.00      0.00       726
         ups       0.00      0.00      0.00       343
         wlk       0.57      0.65      0.61       822

    accuracy                           0.43      3276
   macro avg       0.32      0.32      0.26      3276
weighted avg       0.32      0.43      0.32      3276



/Users/sergio/PycharmProjects/MLEARN/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(clf.predict(X_test), axis=1), target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         dws       0.00      0.00      0.00        75
         jog       1.00      0.19      0.33        77
         sit       0.37      1.00      0.54       210
         std       0.00      0.00      0.00       174
         ups       0.00      0.00      0.00        93
         wlk       0.53      0.68      0.60       190

    accuracy                           0.43       819
   macro avg       0.32      0.31      0.24       819
weighted avg       0.31      0.43      0.31       819

